# Run Moatless Tools

In [1]:
import os

api_key = "alignment-data-generation"
base_url = "https://api.01ww.xyz/v1"

os.environ['OPENAI_API_KEY'] = api_key
os.environ['OPENAI_API_BASE'] = base_url

import openai


First, index the codebase in a vector store.

Set `repo_dir` to the path of the repository you want to index.

In [2]:
from moatless.index import CodeIndex, IndexSettings
from moatless import FileRepository, Workspace

# An OPENAI_API_KEY is required to use the OpenAI Models
model = "gpt-4o-2024-05-13"
index_settings = IndexSettings(
    embed_model="text-embedding-3-small"
    # embed_model="text-embedding-ada-002"
)

repo_dir = "/home/yifan/Code/test_repo"
file_repo = FileRepository(repo_path=repo_dir)

code_index = CodeIndex(file_repo=file_repo, settings=index_settings)
nodes, tokens = code_index.run_ingestion()

print(f"Indexed {nodes} nodes and {tokens} tokens")

workspace = Workspace(file_repo=file_repo, code_index=code_index)

Parsing nodes:   0%|          | 0/41 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/177 [00:00<?, ?it/s]

Indexed 177 nodes and 58057 tokens


In [7]:
res = code_index.find_by_name(class_names=["ContextSpan"])
print(res)

message='Found 1 hits.' hits=[SearchCodeHit(file_path='moatless/file_context.py', spans=[SpanHit(span_id='ContextSpan', rank=0, tokens=17), SpanHit(span_id='ContextSpan.span_id', rank=0, tokens=13), SpanHit(span_id='ContextSpan.dict', rank=0, tokens=14), SpanHit(span_id='ContextSpan.model_dump', rank=0, tokens=23)])]


Then use the `SearchLoop` to find the relevant code.

In [4]:
from moatless.transitions import search_transitions
from moatless import AgenticLoop
import litellm

litellm.set_verbose = False

In [5]:
import logging
logger = logging.getLogger("Loop")
if not logger.handlers:
    print("Logger has no handlers, adding one.")
    # 添加一个handler，例如控制台输出
    console_handler = logging.StreamHandler()
    logger.addHandler(console_handler)
# for handler in logger.handlers:
#     print(handler)
logging.getLogger("Loop").setLevel(logging.INFO)
logger.info("hello world")

hello world


Logger has no handlers, adding one.


In [6]:
instructions = "Remove the token limit check from the completion function"

search_loop = AgenticLoop(
    transition_rules=search_transitions(model=model), 
    workspace=workspace,
    initial_message=instructions,
    prompt_log_dir="../logs",
)

search_response = search_loop.run()

print(workspace.file_context.create_prompt())

Pending:0: Running transition 1. Current total cost: 0
Initializing first state.
Pending:0: Creating state SearchCode with params {'model': 'gpt-4o-2024-05-13', 'id': 1}
Pending:0: Transitioning from Pending to SearchCode
SearchCode:1: Running transition 2. Current total cost: 0
SearchCode:1: Create completion with 2 messages
SearchCode:1: Do completion request to gpt-4o-2024-05-13
SearchCode:1: Received new action Search.
SearchCode:1: Received response with trigger did_search
SearchCode:1: Creating state IdentifyCode with params {'model': 'gpt-4o-2024-05-13', 'ranked_spans': [RankedFileSpan(file_path='moatless/llm/completion.py', span_id='completion', rank=0, tokens=230)], 'id': 2}
SearchCode:1: Transitioning from SearchCode to IdentifyCode
IdentifyCode:2: Running transition 3. Current total cost: 0.01679
IdentifyCode:2: Create completion with 2 messages
IdentifyCode:2: Do completion request to gpt-4o-2024-05-13
IdentifyCode:2: Received new action Identify.
IdentifyCode:2: Received r

moatless/llm/completion.py
```



def completion(
    model: str,
    messages: List,
    max_tokens: int = 1000,
    temperature: float = 0.0,
    trace_name: str = "moatless-agent",
    stop: Optional[List[str]] = None,
    generation_name: Optional[str] = None,
    tools: Optional[List[Dict[str, Any]]] = None,
) -> litellm.ModelResponse:
    if len(messages) == 0:
        raise ValueError("At least one message is required.")

    global _trace_metadata, _mock_response
    metadata = {}
    metadata.update(_trace_metadata)

    if generation_name:
        metadata["generation_name"] = generation_name

    metadata["trace_name"] = trace_name

    tokens = token_counter(messages=messages[-1:])
    if tokens > Settings.max_message_tokens:
        raise ValueError(f"Too many tokens in the new message: {tokens}")

    response = litellm.completion(
        model=model,
        max_tokens=max_tokens,
        temperature=temperature,
        tools=tools,
        stop=stop,
        metadata=

Execute the `CodeLoop` to apply the changes.

In [7]:
from moatless.transitions import code_transitions

litellm.set_verbose=False

code_loop = AgenticLoop(
    transition_rules=code_transitions(model=model),
    workspace=workspace,
    prompt_log_dir="../logs",
)
code_response = code_loop.run(instructions)

print(f"Response: {code_response.message}")

Setting initial message in run is deprecated. Set in contructor.
Pending:0: Running transition 1. Current total cost: 0
Initializing first state.
Using deprecated 'initial_state'. Set initial state in transition_rules instead.
Pending:0: Creating state PlanToCode with params {'model': 'gpt-4o-2024-05-13', 'max_prompt_file_tokens': 16000, 'max_tokens_in_edit_prompt': 500, 'id': 1}
Pending:0: Transitioning from Pending to PlanToCode
PlanToCode:1: Running transition 2. Current total cost: 0
PlanToCode:1: Create completion with 2 messages
PlanToCode:1: Do completion request to gpt-4o-2024-05-13
PlanToCode:1: Received new action ApplyChange.
PlanToCode:1: Received response with trigger edit_code
PlanToCode:1: Creating state EditCode with params {'model': 'gpt-4o-2024-05-13', 'instructions': 'Remove the token limit check from the completion function by deleting the lines that count the tokens and raise a ValueError if the token count exceeds the limit.', 'file_path': 'moatless/llm/completion

Response: The token limit check has been successfully removed from the completion function. The issue has been resolved.


Run a `$ git diff` to see the changes.

In [9]:
import subprocess

output = subprocess.run(
      ["git", "diff"],
      capture_output=True,
      text=True,
      cwd=repo_dir,
)

print(output.stdout)

diff --git a/moatless/llm/completion.py b/moatless/llm/completion.py
index 2c95e47..a926948 100644
--- a/moatless/llm/completion.py
+++ b/moatless/llm/completion.py
@@ -48,10 +48,6 @@ def completion(
 
     metadata["trace_name"] = trace_name
 
-    tokens = token_counter(messages=messages[-1:])
-    if tokens > Settings.max_message_tokens:
-        raise ValueError(f"Too many tokens in the new message: {tokens}")
-
     response = litellm.completion(
         model=model,
         max_tokens=max_tokens,

